In [1]:
import os 
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
tfe.enable_eager_execution()

In [4]:
# TensorFlow MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
dim_hidden = 1024

In [6]:
layer_cnn0 = tf.layers.Conv2D(32, 5, activation = tf.nn.relu)

In [7]:
layer_pool2x2 = tf.layers.MaxPooling2D(2, 2)

In [8]:
layer_cnn1 = tf.layers.Conv2D(64, 5, padding="same", activation = tf.nn.relu)

In [9]:
layer_pool3x3 = tf.layers.MaxPooling2D(3, 3)

In [10]:
layer_cnn2 = tf.layers.Conv2D(64, 5,  padding="same", activation = tf.nn.relu)

In [11]:
layer_pool4x4 = tf.layers.MaxPooling2D(4, 4)

In [12]:
layer_cnn3 = tf.layers.Conv2D(64, 5,  padding="same", activation = tf.nn.relu)

In [13]:
layer_flatten = tf.layers.Flatten()

In [14]:
layer_fc0 = tf.layers.Dense(dim_hidden, activation = tf.nn.relu)

In [15]:
layer_dropout = tf.layers.Dropout(rate=0.80) # dropout rate is 0.75. Retain 0.25

In [16]:
layer_fc1 = tf.layers.Dense(10, activation = tf.nn.relu) # 1

In [17]:
# forward propagation
def prediction(X, training):
    values = tf.constant(X)
    values = layer_pool4x4(values) # this must be the first layer
    values = layer_cnn0(values)
    values = layer_cnn1(values)
    values = layer_cnn2(values)
    values = layer_cnn3(values)
    values = layer_pool3x3(values)
    values = layer_flatten(values)
    values = layer_fc0(values)
    values = layer_dropout(values, training=training)
    values = layer_fc1(values)
    return values

In [18]:
# cross entropy loss
def loss(X, y, training):
    logits = prediction(X, training)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = logits)
    loss = tf.reduce_mean(loss)
    return loss


In [19]:
def binary_accuracy(X, y):
    logits = prediction(X, training = False)
    predict = tf.argmax(logits, 1).numpy()
    target = np.argmax(y, 1)
    binary_accuracy = np.sum(predict == target)/len(target)
    return(binary_accuracy)

In [20]:
X_validation = mnist.validation.images
y_validation = mnist.validation.labels
X_validation = X_validation.reshape([-1,28,28,1])

In [21]:
def v_binary_accuracy() :
    return(binary_accuracy(X_validation, y_validation))

In [22]:
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3, epsilon=1e-9)

In [23]:
batch_size = 50
iters = 2000

In [24]:
for i in range(iters):
    X, y = mnist.train.next_batch(batch_size)
    X = X.reshape([-1,28,28,1])
    optimizer.minimize(lambda: loss(X, y, True))

    if i % 100 == 0:
        batch_accuracy = binary_accuracy(X, y)
        validation_accuracy = v_binary_accuracy()
        print("batch %d, batch accuracy %.3f;  validation accuracy %.3f" %
                                (i, batch_accuracy, validation_accuracy))

batch 0, batch accuracy 0.220;  validation accuracy 0.148
batch 100, batch accuracy 0.480;  validation accuracy 0.420
batch 200, batch accuracy 0.620;  validation accuracy 0.662
batch 300, batch accuracy 0.960;  validation accuracy 0.779
batch 400, batch accuracy 0.820;  validation accuracy 0.799
batch 500, batch accuracy 0.840;  validation accuracy 0.797
batch 600, batch accuracy 0.900;  validation accuracy 0.803
batch 700, batch accuracy 0.920;  validation accuracy 0.874
batch 800, batch accuracy 0.860;  validation accuracy 0.881
batch 900, batch accuracy 0.940;  validation accuracy 0.883
batch 1000, batch accuracy 0.800;  validation accuracy 0.893
batch 1100, batch accuracy 0.940;  validation accuracy 0.897
batch 1200, batch accuracy 0.920;  validation accuracy 0.889
batch 1300, batch accuracy 0.940;  validation accuracy 0.898
batch 1400, batch accuracy 0.920;  validation accuracy 0.902
batch 1500, batch accuracy 0.920;  validation accuracy 0.901
batch 1600, batch accuracy 0.840;  v

In [25]:
# evaluate the result
X, y = mnist.test.images, mnist.test.labels
X = X.reshape([-1,28,28,1])
test_accuracy = binary_accuracy(X, y)
print("test accuracy %g" % (test_accuracy))

test accuracy 0.8864
